In [1]:
# Librerías estándar y de terceros
import os
import sys
import gc
import math as m
import numpy as np
import scipy
import pandas as pd
from scipy import *
from scipy.signal import butter, filtfilt, hilbert
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot as plt

# Configuración de rutas para importaciones personalizadas
sys.path.append('/projects/DEIKE/cmartinb/jupyter_notebook/project_specific/turbulence')
sys.path.append('/projects/DEIKE/cmartinb/functions')
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from mpl_toolkits.axes_grid1 import make_axes_locatable
# Funciones y clases personalizadas
from prepare import load_object, save_object, field
from defs import Case, Interface2D
from phase import extract_phase
from funciones import *  # Asegúrate de que la importación con * sea necesaria, a veces es mejor importar solo las funciones necesarias

os.chdir('/projects/DEIKE/cmartinb/')

In [ ]:
def read_surface(filename, N):
    val = np.fromfile(filename, dtype=np.float32)
    val = val.reshape(N,N)
    return val

In [2]:
# DEFINE CASE SO WE CAN AUTHOMATIZE IT 
kpHs = '0p16' # 0p16 
uoc = '0p50' # 0p50 0p75
reW = '2.5e4' #1.0e5
reA = 720
#maxLevel = 10
maxLevel = 10
Bo=1000 # 25, 1000

rho1=1
rho2=1*10**(-3)

if reW == '1.0e5':
    Re_water = 1 * 10**5
else:
    Re_water = 2.5 * 10**4

if maxLevel == 11:
    N = 1024
else:
    N = 512 

#Common parameters 
kp = 4
u = 0.25
lambdap = 2*m.pi/kp

L0 = 2*np.pi;

ak,  c, omegap, nu_water, g, uoc_val, sigma, Tp= calculate_parameters(kpHs, uoc, u, kp, Bo, rho1, rho2)
print('ak is', ak, 'c is' ,c, 'w_p$ is', omegap, '$\nu_w$ is' , nu_water, 'g is' ,g, 'N', N)

ak is 0.16 c is 0.5 w_p$ is 2.0 $
u_w$ is 3.141592653589793e-05 g is 0.999001997004992 N 512


In [43]:
def read_surface(filename, N):
    val = np.fromfile(filename, dtype=np.float32)
    val = val.reshape(N,N)
    return val

In [44]:
#work_dir = f'/projects/DEIKE/nscapin/broadband_reorder/re{reA}_bo0{Bo}_P{kpHs}_uoc{uoc}_reW{reW}_L{maxLevel}/'
#for Bo=1000 I run the simulation
#work_dir = f'/projects/DEIKE/cmartinb/cases_multiphase_broadbanded/re{reA}_bo{Bo}_kpHs{kpHs}_uoc{uoc}_reW{reW}_L{maxLevel}/'
work_dir = '/projects/DEIKE/cmartinb/Ruidata/surface_kpHs0p3_dt0p2_kp0p157_g9p8/'
#data = np.loadtxt(work_dir+'eta/global_int.out')
#re720_bo0200_P0p16_uoc0p50_reW1.0e5_L10
#istep_c =data[:, 1]
#time = data[:,0] 
#print(istep_c)
#print(time.shape)

In [45]:
val = read_surface(work_dir + '/eta_matrix_000000000', N)
print(val.shape)

(1024, 1024)


In [47]:
import re 
N = 1024

eta_series = np.zeros((istep_c.shape[0],N,N), dtype=np.float32)
j=0
#val = read_surface(work_dir + '/eta_matrix_000000000', N)

pattern = re.compile(r'eta_matrix_(\d+)$')
files = []
for fname in os.listdir(work_dir):
    m = pattern.match(fname)
    if m:
        files.append((int(m.group(1)), fname))
# Ordeno por el número de iteración (el grupo (\d+))
files.sort(key=lambda x: x[0])


# --- Reservo el array 3D para almacenar la serie ---
n_steps = len(files)
eta_series = np.empty((n_steps, N, N), dtype= np.float32  )

# --- Bucle de lectura ---
for idx, (_, fname) in enumerate(files):
    fullpath = os.path.join(work_dir, fname)
    # Si es binario:
    print(fullpath)
    data = read_surface(fullpath,N)
    eta_series[idx, :, :] = data
    print(f'Leído step {idx:03d} ← {fname}')


/projects/DEIKE/cmartinb/Ruidata/surface_kpHs0p3_dt0p2_kp0p157_g9p8/eta_matrix_000000000
Leído step 000 ← eta_matrix_000000000
/projects/DEIKE/cmartinb/Ruidata/surface_kpHs0p3_dt0p2_kp0p157_g9p8/eta_matrix_000000021
Leído step 001 ← eta_matrix_000000021
/projects/DEIKE/cmartinb/Ruidata/surface_kpHs0p3_dt0p2_kp0p157_g9p8/eta_matrix_000000049
Leído step 002 ← eta_matrix_000000049
/projects/DEIKE/cmartinb/Ruidata/surface_kpHs0p3_dt0p2_kp0p157_g9p8/eta_matrix_000000084
Leído step 003 ← eta_matrix_000000084
/projects/DEIKE/cmartinb/Ruidata/surface_kpHs0p3_dt0p2_kp0p157_g9p8/eta_matrix_000000121
Leído step 004 ← eta_matrix_000000121
/projects/DEIKE/cmartinb/Ruidata/surface_kpHs0p3_dt0p2_kp0p157_g9p8/eta_matrix_000000157
Leído step 005 ← eta_matrix_000000157
/projects/DEIKE/cmartinb/Ruidata/surface_kpHs0p3_dt0p2_kp0p157_g9p8/eta_matrix_000000193
Leído step 006 ← eta_matrix_000000193
/projects/DEIKE/cmartinb/Ruidata/surface_kpHs0p3_dt0p2_kp0p157_g9p8/eta_matrix_000000229
Leído step 007 ← eta_m

'\n\nfor i in istep_c:\n    filename = work_dir + f\'eta/eta_loc/eta_loc_t{int(i):09}.bin\'\n    etalo = np.fromfile(filename)\n    size  = etalo.shape;\n    tot_row = 18\n    tot_row_i = int(size[0]/tot_row);\n    etalo = etalo.reshape([tot_row_i, tot_row]);\n    \n    eta_m0  = 1.0; cirp_th = 0.20;\n    new_row = 0;\n    for i in range(tot_row_i):\n        if ( abs(etalo[i][12]-eta_m0) < cirp_th ):\n            new_row += 1;\n    #\n    print(\'iteration\', j ,"Second pass of remove")\n    etal = np.zeros([new_row, 18]);\n    for i in range(new_row):\n        if ( abs(etalo[i][12]-eta_m0) < cirp_th ):\n            etal[i][:] = etalo[i][:];\n    xarray = np.linspace(-L0/2., L0/2.,N,endpoint=False)+L0/2/N/2 # Centered grid for interpolation\n    yarray = np.linspace(-L0/2., L0/2.,N,endpoint=False)+L0/2/N/2 # Centered grid for interpolation\n    xtile, ytile = np.meshgrid(xarray,yarray)\n    #print(etal[:,0])\n    eta = griddata((etal[:,0].ravel(), etal[:,1].ravel()), etal[:,12].ravel()

In [50]:
#np.save(f'/projects/DEIKE/cmartinb/eta/rui_eta_series_re{reA}_bo0{Bo}_P{kpHs}_uoc{uoc}_reW{reW}_L{maxLevel}.npy', eta_series)
import h5py

with h5py.File(f'/projects/DEIKE/cmartinb/eta/rui_eta_series_kpHs0p3_dt0p2_kp0p157_g9p8.h5','w') as f:
    f.create_dataset('eta', data=eta_series, compression='gzip')

In [49]:
print(f'/projects/DEIKE/cmartinb/eta/rui_eta_series_kpHs0p3_dt0p2_kp0p157_g9p8.h5')

/projects/DEIKE/cmartinb/eta/rui_eta_series_re720_bo01000_P0p16_uoc0p50_reW2.5e4_L10.h5
